In [1]:
import numpy as np

# imports custom RNCRN functions that are used throughout multiple  notebooks 
import os, sys
from pathlib import Path

module_path = str(Path(os.path.abspath('')).parent)
if module_path not in sys.path:
    sys.path.append(module_path)

from functions.target_systems import uni_stable, cubic_bi_stable
from functions.RNCRN_train import train_static_executive_species, initialize_single_RNCRN, save_mat_model_static_exec
from functions.RNCRN_phase_plane import merge_multi_state_data
from functions.RNCRN_phase_line import create_training_data_1D_from_ODE
from functions.create_chimera_tools import initialize_static_execs_data, initialize_single_RNCRN_static_exec, merge_static_execs_multi_state_data

In [2]:
# sets the parameters of the target dynamics 
target_dynamics_a = lambda x: uni_stable(x, a=5, b=6)
target_dynamics_b = lambda x: cubic_bi_stable(x, r1=2,r2=5, r3=8)

In [3]:
# sets intervals of executive species compact subsets (i.e. K_1, K_2)
lower_limit = 1
upper_limit = 9
step_size = 0.01 

# creates training data from ODE
inputs_a, targets_a = create_training_data_1D_from_ODE(target_dynamics_a, lower_limit, upper_limit, step_size )
inputs_b, targets_b = create_training_data_1D_from_ODE(target_dynamics_b, lower_limit, upper_limit, step_size )

# defines the number of parameters
number_of_static_exec_species = 1

# defines map between parameter species value and target dynamics
static_exec_state_a = [1]
static_exec_state_b = [0]

# initialize training data for the parameter species 
exec_inputs_a = initialize_static_execs_data(number_of_static_exec_species, inputs_a, static_exec_state_a)
exec_inputs_b = initialize_static_execs_data(number_of_static_exec_species, inputs_b, static_exec_state_b)

In [4]:
# sets the hyper-structure of the RNCRN (i.e. N and M)   
number_of_exec_species = 1
number_of_chemical_perceptrons = 2

# calls a function that randomly initializes the RNCRN for training with automatic differentiation  
rnd_seed = 2
params = initialize_single_RNCRN_static_exec(number_of_exec_species, number_of_chemical_perceptrons, number_of_static_exec_species, rnd_seed)

In [5]:
# merges data into the correct format for training  
list_of_inputs = [inputs_a, inputs_b]
list_of_targets = [targets_a, targets_b]
list_of_exec_inputs = [exec_inputs_a, exec_inputs_b]

inputs, targets, exec_inputs = merge_static_execs_multi_state_data(list_of_inputs, list_of_targets, list_of_exec_inputs)

In [6]:
# call training function
start_learning_rate = 5e1
batch_size = 500
error_threshold = 0.05 
number_of_epochs = 1000

loss, params, lowest_params, lowest_loss = train_static_executive_species(inputs, targets, exec_inputs, params, number_of_epochs, start_learning_rate, batch_size, error_threshold)

Epoch: 500 loss =  144.60506
Lowest loss found!


In [7]:
save_mat_model_static_exec(params, number_of_chemical_perceptrons, number_of_static_exec_species, number_of_exec_species, loss, rnd_seed, flag='toogle' )